# Simulation with traffic generation.

This demo is for the case of extrapolating the maximum effect caused by a regional traffic flow. 

The bridge: 

- Has four 3.5m-width lanes. 
- Has a length of 20m.
- Has a width of 16m.
- Has one load effect being considered. 

The traffic flow: 

- Vehicles are generated from Garage model. 
- Headways are in freeflow condition (Poisson arrival model). 



---

The part that has already been introduced in the single_vehicle demo. 

But this time we change to define a 4-lane bridge. 

In [1]:
import pybtls as pb
from pathlib import Path


# set the load effect by using a 2D influence surface
lanes_position = [(0.5, 4.0), (4.0, 7.5), (8.5, 12.0), (12.0, 15.5)]
IS_matrix = [
    [0.0, 0.0, 4.0, 8.0, 12.0, 16.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [10.0, 0.0, 2.5, 5.0, 2.5, 0.0],
    [20.0, 0.0, 0.0, 0.0, 0.0, 0.0],
]
load_effect = pb.InfluenceSurface()
load_effect.set_IS(IS_matrix, lanes_position)

# set the bridge
bridge = pb.Bridge(length=20.0, no_lane=4)
bridge.add_load_effect(inf_line_surf=load_effect, threshold=0.0)
bridge.add_load_effect(
    inf_line_surf=load_effect, threshold=1000.0
)  # We add the same load effect again, but this time with a threshold of 1000.0 for the later POT analysis

Set lane flow compositions (here we just use the same flow for all lanes). 

For the lane_dir, 1 means from left to right and 2 means from right to left, 
based on the same coordinate system as the influence surface. 

In [2]:
lfc_lane1 = pb.LaneFlowComposition(lane_index=1, lane_dir=1)
lfc_lane1.assign_lane_data(
    hourly_truck_flow=[100] * 24,
    hourly_car_flow=[0] * 24,
    hourly_speed_mean=[80 / 3.6 * 10] * 24,
    hourly_speed_std=[10.0] * 24,
)

lfc_lane2 = pb.LaneFlowComposition(lane_index=2, lane_dir=1)
lfc_lane2.assign_lane_data(
    hourly_truck_flow=[100] * 24,
    hourly_car_flow=[0] * 24,
    hourly_speed_mean=[80 / 3.6 * 10] * 24,
    hourly_speed_std=[10.0] * 24,
)

lfc_lane3 = pb.LaneFlowComposition(lane_index=3, lane_dir=2)
lfc_lane3.assign_lane_data(
    hourly_truck_flow=[100] * 24,
    hourly_car_flow=[0] * 24,
    hourly_speed_mean=[80 / 3.6 * 10] * 24,
    hourly_speed_std=[10.0] * 24,
)

lfc_lane4 = pb.LaneFlowComposition(lane_index=4, lane_dir=2)
lfc_lane4.assign_lane_data(
    hourly_truck_flow=[100] * 24,
    hourly_car_flow=[0] * 24,
    hourly_speed_mean=[80 / 3.6 * 10] * 24,
    hourly_speed_std=[10.0] * 24,
)

Set vehicle generator. Here we use the garage one.


In [3]:
# The kernel is to ensure variation between the generated vehicles and garages.
kernel = [[1.0, 0.08], [1.0, 0.05], [1.0, 0.02]]

vehicle_gen = pb.VehicleGenGarage(
    garage=Path(".") / "garage.txt", kernel=kernel, garage_format=4
)

Choose headway generator with the freeflow one.

In [4]:
# set headway generator
headway_gen = pb.HeadwayGenFreeflow()

Assemble the vehicle generator and the headway generator with the lfc information to get a traffic generator.

In [5]:
traffic_gen = pb.TrafficGenerator(no_lane=4)
traffic_gen.add_lane(vehicle_gen=vehicle_gen, headway_gen=headway_gen, lfc=lfc_lane1)
traffic_gen.add_lane(vehicle_gen=vehicle_gen, headway_gen=headway_gen, lfc=lfc_lane2)
traffic_gen.add_lane(vehicle_gen=vehicle_gen, headway_gen=headway_gen, lfc=lfc_lane3)
traffic_gen.add_lane(vehicle_gen=vehicle_gen, headway_gen=headway_gen, lfc=lfc_lane4)
# traffic_gen.set_start_time(0.0)  # optional, the default start_time is 0.0 unless you want to change it.

Set output, which will be written to HDD. 

To do the extrapolation, we need the block-max and peak-over-threshold output. 

In [6]:
output_config = pb.OutputConfig()
output_config.set_event_output(write_time_history=True, write_each_event=True)
output_config.set_BM_output(write_vehicle=True, write_summary=True, write_mixed=True)
output_config.set_POT_output(write_vehicle=True, write_summary=True, write_counter=True)
output_config.set_fatigue_output(
    write_rainflow_output=True, rainflow_decimal=0, rainflow_cut_off=100.0
)

Set the simulation and run. 

In [7]:
sim_task = pb.Simulation(Path(".") / "temp")
sim_task.add_sim(
    bridge=bridge,
    traffic=traffic_gen,
    no_day=250,  # 1-year working days
    output_config=output_config,
    time_step=0.1,
    min_gvw=35,  # Ignore vehicles with GVW less than 35 kN.
    # active_lane=[1,2,3,4],  # optional, if not set, all lanes will be active.
    # track_progress=False,  # optional, if True, the progress print will show up.
    tag="Case2",
)

# run simulation
sim_task.run(no_core=1)


Bridge 20 m: Flushing AllEvents buffer: 10000 events at 7/1/0 18:19:26	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 14/1/0 12:52:9	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 21/1/0 3:23:48	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 2/2/0 21:13:43	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 9/2/0 13:39:1	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 16/2/0 6:27:29	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 22/2/0 19:17:1	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 4/3/0 11:17:34	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 11/3/0 1:15:4	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 17/3/0 18:17:23	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 24/3/0 8:4:23	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 6/4/0 1:46:42	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 12/4/0 17:3:15	
Bridge 20 m: Flushing AllEvents buffer: 10000 events at 19/4/0 9:2

Check what are the outputs.

In [8]:
sim_output = sim_task.get_output()
print(sim_output.keys())

dict_keys(['Case2'])


Use two simple loops to display all the read-in outputs. 

The data are stored as pandas.DataFrame.

In [9]:
from IPython.display import display

case2_output = sim_output["Case2"]

for i in case2_output.get_summary():
    print(i)

    for j in case2_output.read_data(i):
        print(j)
        display(case2_output.read_data(i)[j])

time_history
TH_20


,Time (s),No. Trucks,Effect 1,Effect 2
0,4.258400e+01,1,0.000,0.000
1,4.268400e+01,1,12.384,12.384
2,4.278400e+01,1,30.136,30.136
3,4.288400e+01,1,58.898,58.898
4,4.298400e+01,1,87.660,87.660
...,...,...,...,...
4702994,2.159993e+07,1,66.352,66.352
4702995,2.159993e+07,1,45.311,45.311
4702996,2.159993e+07,1,29.782,29.782
4702997,2.159993e+07,1,14.254,14.254


all_events
BL_20_AllEvents


,Time (s),No. Trucks,Effect 1,Effect 2
0,4.258360e+01,1,110.7130,110.7130
1,1.583170e+02,1,138.2720,138.2720
2,3.166630e+02,1,44.7878,44.7878
3,3.459510e+02,1,318.2000,318.2000
4,3.474340e+02,1,33.4742,33.4742
...,...,...,...,...
375141,2.159990e+07,1,813.5680,813.5680
375142,2.159990e+07,1,847.5070,847.5070
375143,2.159990e+07,1,137.7530,137.7530
375144,2.159990e+07,1,18.6105,18.6105


BM_by_no_trucks
BM_V_20_3


...

,Block,Effect 1,Effect 2
0,1,1453,4
1,2,1498,12
2,3,1455,12
3,4,1570,11
4,5,1424,8
...,...,...,...
282,247,1541,4
283,248,1313,6
284,248,198,0
285,249,1497,10


fatigue_rainflow
RC_20_1


,Amplitude,No. Cycles
0,100,1169.5
1,101,1229.5
2,102,1306.0
3,103,1416.0
4,104,1474.5
...,...,...
1101,1357,0.5
1102,1385,0.5
1103,1423,0.5
1104,1471,0.5


RC_20_2


,Amplitude,No. Cycles
0,100,1169.5
1,101,1229.5
2,102,1306.0
3,103,1416.0
4,104,1474.5
...,...,...
1101,1357,0.5
1102,1385,0.5
1103,1423,0.5
1104,1471,0.5


The simulation output can be saved and loaded for further analysis.

In [ ]:
pb.save_output(sim_output, Path("./temp") / "output.pkl")
sim_output = pb.load_output(Path("./temp") / "output.pkl")

Outputs have been successfully loaded from temp/output.pkl!
